In [19]:
import pandas as pd
from sklearn.metrics import auc, roc_curve
import pandasql as pdsql
import sas7bdat as sas
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
pd.set_option('display.max_columns', 500)
%matplotlib inline

In [85]:
def dataframe_with_null_dummies(df):
    '''
    INPUT: dataframe
    OUTPUT: dataframe with dummy columns for IS_NULL for all columns that contain null values'''
    new_df = df.copy()
    for col in new_df.columns:
        new_df[f'{col}_IS_NULL'] = new_df[col].isna()
    return new_df

def get_dummy_dataframe_for_categorical(df, dummy_column_type='O'):
    '''
    INPUT: dataframe
    OUTPUT: dataframe with dummy columns added for all of X type
    columns'''
    new_df = df.copy()
    dtypes = list(df.dtypes.values)
    for i in range(len(dtypes)):
        if dtypes[i] == dummy_column_type:
            dum_col_to_add = pd.get_dummies(df.iloc[:,i],prefix='DUM_')
            new_df = pd.concat([new_df, dum_col_to_add], axis=1)
    return new_df

def get_valid_columns(df, valid_types=['bool', 'uint8', 'int64', 'float64']):
    dtypes = list(df.dtypes.values)
    valid_columns = []
    for i in range(len(dtypes)):
        if dtypes[i] in valid_types:
            valid_columns.append(i)
    return valid_columns

def plot_roc(model, x_columns, y_true, title="model type"):
    '''
    INPUT: fitted model, array of x values, array of target values, optional:
    title of outputted figure
    OUTPUT: ROC curve with AUC value
    '''

    y_pred = model.predict_proba(x_columns)

    fpr, tpr, threshold = roc_curve(y_true, y_pred[:, 1])
    area_under_curve = auc(fpr, tpr)

    # method I: plt
    fig, ax = plt.subplots()
    if title == "model type":
        model_name = str(type(model)).split('.')[-1].strip(">\'")
    else:
        model_name = title

    plt.title(f'{model_name} ROC')
    ax.plot(fpr, tpr, 'k', label='C AUC = %0.3f' % area_under_curve)

    ax.legend(loc='lower right')
    ax.plot([0, 1], [0, 1], 'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

In [92]:
def all_df_featurization(train_df, test_df, target_name, other_columns_to_ignore):
    x_column_set = list(set(df_application_train).intersection(df_application_test)\
                     - set(other_columns_to_ignore))
    train_df = train_df[x_column_set].copy()
    test_df = test_df[x_column_set].copy()
    
    clean_train_df = dataframe_with_null_dummies(train_df)
    clean_test_df = dataframe_with_null_dummies(test_df)
    
    
    clean_train_df = get_dummy_dataframe_for_categorical(clean_train_df)
    clean_test_df = get_dummy_dataframe_for_categorical(clean_test_df)
    
    val_columns = set(clean_train_df.columns).intersection(clean_test_df.columns)
    clean_train_df = clean_train_df.iloc[:,val_columns].copy()
    clean_test_df = clean_test_df.iloc[:,val_columns].copy()
    
    clean_train_mean = clean_train_df.mean()
    clean_test_mean = clean_test_df.mean()
    
    for col in clean_df.columns:
        clean_train_df[col] = clean_train_df[col].fillna(clean_mean[col])
        clean_test_df[col] = clean_test_df[col].fillna(clean_test_mean[col])
        
    return clean_train_df, clean_test_df

In [91]:
!ls '/Users/ElliottC/.kaggle/competitions/home-credit-default-risk/csvs/'

file_path = '/Users/ElliottC/.kaggle/competitions/home-credit-default-risk/csvs/'

df_application_train = pd.read_csv(file_path + 'application_train.csv')
df_application_test = pd.read_csv(file_path + 'application_test.csv')

HomeCredit_columns_description.csv bureau_balance.csv
POS_CASH_balance.csv               credit_card_balance.csv
application_test.csv               installments_payments.csv
application_train.csv              previous_application.csv
bureau.csv                         sample_submission.csv


In [95]:
train_df, test_df = all_df_featurization(df_application_train, df_application_test, 'TARGET', ['SK_ID_CURR'])

TypeError: len() of unsized object

In [65]:
columns

In [56]:
testing_df = all_df_featurization(df_application_test, 'test')[columns.remove('TARGET')]

AttributeError: 'NoneType' object has no attribute 'remove'

In [33]:
model = RandomForestClassifier()
model.fit(training_df.drop('TARGET', axis=1), training_df['TARGET'])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [94]:
df_application_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,0.9851,0.8040,0.0497,0.0806,0.0345,0.2917,0.3333,0.0128,0.079,0.0554,0.0,0.0,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,Government,NaN,0.555912,0.729567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [35]:
list(set(training_df.columns) - set(testing_df.columns))

['DUM__Maternity leave', 'DUM__Unknown', 'TARGET']

In [34]:
model.predict(testing_df)

ValueError: Number of features of the model must match the input. Model n_features is 365 and input n_features is 362 

In [127]:
clean_df = dataframe_with_null_dummies(clean_df)

In [128]:
clean_df = get_dummy_dataframe_for_categorical(clean_df)

In [129]:
clean_df.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,AMT_ANNUITY_IS_NULL,AMT_GOODS_PRICE_IS_NULL,NAME_TYPE_SUITE_IS_NULL,OWN_CAR_AGE_IS_NULL,OCCUPATION_TYPE_IS_NULL,CNT_FAM_MEMBERS_IS_NULL,EXT_SOURCE_1_IS_NULL,EXT_SOURCE_2_IS_NULL,EXT_SOURCE_3_IS_NULL,APARTMENTS_AVG_IS_NULL,BASEMENTAREA_AVG_IS_NULL,YEARS_BEGINEXPLUATATION_AVG_IS_NULL,YEARS_BUILD_AVG_IS_NULL,COMMONAREA_AVG_IS_NULL,ELEVATORS_AVG_IS_NULL,ENTRANCES_AVG_IS_NULL,FLOORSMAX_AVG_IS_NULL,FLOORSMIN_AVG_IS_NULL,LANDAREA_AVG_IS_NULL,LIVINGAPARTMENTS_AVG_IS_NULL,LIVINGAREA_AVG_IS_NULL,NONLIVINGAPARTMENTS_AVG_IS_NULL,NONLIVINGAREA_AVG_IS_NULL,APARTMENTS_MODE_IS_NULL,BASEMENTAREA_MODE_IS_NULL,YEARS_BEGINEXPLUATATION_MODE_IS_NULL,YEARS_BUILD_MODE_IS_NULL,COMMONAREA_MODE_IS_NULL,ELEVATORS_MODE_IS_NULL,ENTRANCES_MODE_IS_NULL,FLOORSMAX_MODE_IS_NULL,FLOORSMIN_MODE_IS_NULL,LANDAREA_MODE_IS_NULL,LIVINGAPARTMENTS_MODE_IS_NULL,LIVINGAREA_MODE_IS_NULL,NONLIVINGAPARTMENTS_MODE_IS_NULL,NONLIVINGAREA_MODE_IS_NULL,APARTMENTS_MEDI_IS_NULL,BASEMENTAREA_MEDI_IS_NULL,YEARS_BEGINEXPLUATATION_MEDI_IS_NULL,YEARS_BUILD_MEDI_IS_NULL,COMMONAREA_MEDI_IS_NULL,ELEVATORS_MEDI_IS_NULL,ENTRANCES_MEDI_IS_NULL,FLOORSMAX_MEDI_IS_NULL,FLOORSMIN_MEDI_IS_NULL,LANDAREA_MEDI_IS_NULL,LIVINGAPARTMENTS_MEDI_IS_NULL,LIVINGAREA_MEDI_IS_NULL,NONLIVINGAPARTMENTS_MEDI_IS_NULL,NONLIVINGAREA_MEDI_IS_NULL,FONDKAPREMONT_MODE_IS_NULL,HOUSETYPE_MODE_IS_NULL,TOTALAREA_MODE_IS_NULL,WALLSMATERIAL_MODE_IS_NULL,EMERGENCYSTATE_MODE_IS_NULL,OBS_30_CNT_SOCIAL_CIRCLE_IS_NULL,DEF_30_CNT_SOCIAL_CIRCLE_IS_NULL,OBS_60_CNT_SOCIAL_CIRCLE_IS_NULL,DEF_60_CNT_SOCIAL_CIRCLE_IS_NULL,DAYS_LAST_PHONE_CHANGE_IS_NULL,AMT_REQ_CREDIT_BUREAU_HOUR_IS_NULL,AMT_REQ_CREDIT_BUREAU_DAY_IS_NULL,AMT_REQ_CREDIT_BUREAU_WEEK_IS_NULL,AMT_REQ_CREDIT_BUREAU_MON_IS_NULL,AMT_REQ_CREDIT_BUREAU_QRT_IS_NULL,AMT_REQ_CREDIT_BUREAU_YEAR_IS_NULL,DUM__Cash loans,DUM__Revolving

In [ ]:
clean_df = clean_df.fillna(clean_df.mean())
val_columns = get_valid_columns(clean_df)
return clean_df, val_columns

In [121]:
df_application_test.fillna(df_application_test.mean())

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,Unaccompanied,Working,Higher education,Married,House / apartment,0.018850,-19241,-2329,-5170.0,-812,11.786027,1,1,0,1,0,1,NaN,2.0,2,2,TUESDAY,18,0,0,0,0,0,0,Kindergarten,0.752614,0.789654,0.159520,0.066000,0.059000,0.973200,0.751137,0.047624,0.085168,0.137900,0.125000,0.238423,0.067192,0.105885,0.050500,0.009231,0.029387,0.067200,0.061200,0.973200,0.758327,0.045223,0.08057,0.137900,0.12500,0.233854,0.065914,0.110874,0.052600,0.008358,0.028161,0.066600,0.059000,0.973200,0.754344,0.04742,0.084128,0.1379,0.125000,0.237846,0.068069,0.107063,0.051400,0.008979,0.029296,NaN,block of flats,0.039200,"Stone, brick",No,0.0,0.0,0.0,0.0,-1740.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.035792,-18064,-4469,-9118.0,-1623,11.786027,1,1,0,1,0,0,Low-skill Laborers,2.0,2,2,FRIDAY,9,0,0,0,0,0,0,Self-employed,0.564990,0.291656,0.432962,0.122388,0.090065,0.978828,0.751137,0.047624,0.085168,0.151777,0.233706,0.238423,0.067192,0.105885,0.112286,0.009231,0.029387,0.119078,0.088998,0.978292,0.758327,0.045223,0.08057,0.147161,0.22939,0.233854,0.065914,0.110874,0.110687,0.008358,0.028161,0.122809,0.089529,0.978822,0.754344,0.04742,0.084128,0.1512,0.233154,0.237846,0.068069,0.107063,0.113368,0.008979,0.029296,NaN,NaN,0.107129,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,NaN,Working,Higher education,Married,House / apartment,0.019101,-20038,-4458,-2175.0,-3503,5.000000,1,1,0,1,0,0,Drivers,2.0,2,2,MONDAY,14,0,0,0,0,

In [140]:
cleaned_df = all_df_featurization(df_application_test)

In [142]:
model.predict(cleaned_df)

ValueError: Number of features of the model must match the input. Model n_features is 171 and input n_features is 309 

In [138]:
cleaned_df.mean()['OWN_CAR_AGE']

11.786027263875365

In [116]:
model.predict(cleaned_df.iloc[:,valid_columns])

ValueError: Number of features of the model must match the input. Model n_features is 171 and input n_features is 309 

In [114]:
cleaned_df

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,AMT_ANNUITY_IS_NULL,AMT_GOODS_PRICE_IS_NULL,NAME_TYPE_SUITE_IS_NULL,OWN_CAR_AGE_IS_NULL,OCCUPATION_TYPE_IS_NULL,CNT_FAM_MEMBERS_IS_NULL,EXT_SOURCE_1_IS_NULL,EXT_SOURCE_2_IS_NULL,EXT_SOURCE_3_IS_NULL,APARTMENTS_AVG_IS_NULL,BASEMENTAREA_AVG_IS_NULL,YEARS_BEGINEXPLUATATION_AVG_IS_NULL,YEARS_BUILD_AVG_IS_NULL,COMMONAREA_AVG_IS_NULL,ELEVATORS_AVG_IS_NULL,ENTRANCES_AVG_IS_NULL,FLOORSMAX_AVG_IS_NULL,FLOORSMIN_AVG_IS_NULL,LANDAREA_AVG_IS_NULL,LIVINGAPARTMENTS_AVG_IS_NULL,LIVINGAREA_AVG_IS_NULL,NONLIVINGAPARTMENTS_AVG_IS_NULL,NONLIVINGAREA_AVG_IS_NULL,APARTMENTS_MODE_IS_NULL,BASEMENTAREA_MODE_IS_NULL,YEARS_BEGINEXPLUATATION_MODE_IS_NULL,YEARS_BUILD_MODE_IS_NULL,COMMONAREA_MODE_IS_NULL,ELEVATORS_MODE_IS_NULL,ENTRANCES_MODE_IS_NULL,FLOORSMAX_MODE_IS_NULL,FLOORSMIN_MODE_IS_NULL,LANDAREA_MODE_IS_NULL,LIVINGAPARTMENTS_MODE_IS_NULL,LIVINGAREA_MODE_IS_NULL,NONLIVINGAPARTMENTS_MODE_IS_NULL,NONLIVINGAREA_MODE_IS_NULL,APARTMENTS_MEDI_IS_NULL,BASEMENTAREA_MEDI_IS_NULL,YEARS_BEGINEXPLUATATION_MEDI_IS_NULL,YEARS_BUILD_MEDI_IS_NULL,COMMONAREA_MEDI_IS_NULL,ELEVATORS_MEDI_IS_NULL,ENTRANCES_MEDI_IS_NULL,FLOORSMAX_MEDI_IS_NULL,FLOORSMIN_MEDI_IS_NULL,LANDAREA_MEDI_IS_NULL,LIVINGAPARTMENTS_MEDI_IS_NULL,LIVINGAREA_MEDI_IS_NULL,NONLIVINGAPARTMENTS_MEDI_IS_NULL,NONLIVINGAREA_MEDI_IS_NULL,FONDKAPREMONT_MODE_IS_NULL,HOUSETYPE_MODE_IS_NULL,TOTALAREA_MODE_IS_NULL,WALLSMATERIAL_MODE_IS_NULL,EMERGENCYSTATE_MODE_IS_NULL,OBS_30_CNT_SOCIAL_CIRCLE_IS_NULL,DEF_30_CNT_SOCIAL_CIRCLE_IS_NULL,OBS_60_CNT_SOCIAL_CIRCLE_IS_NULL,DEF_60_CNT_SOCIAL_CIRCLE_IS_NULL,DAYS_LAST_PHONE_CHANGE_IS_NULL,AMT_REQ_CREDIT_BUREAU_HOUR_IS_NULL,AMT_REQ_CREDIT_BUREAU_DAY_IS_NULL,AMT_REQ_CREDIT_BUREAU_WEEK_IS_NULL,AMT_REQ_CREDIT_BUREAU_MON_IS_NULL,AMT_REQ_CREDIT_BUREAU_QRT_IS_NULL,AMT_REQ_CREDIT_BUREAU_YEAR_IS_NULL,DUM__Cash loans,DUM__Revolving

In [80]:
df_application_test.fillna(df_application_test.mean())

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,Unaccompanied,Working,Higher education,Married,House / apartment,0.018850,-19241,-2329,-5170.0,-812,11.786027,1,1,0,1,0,1,NaN,2.0,2,2,TUESDAY,18,0,0,0,0,0,0,Kindergarten,0.752614,0.789654,0.159520,0.066000,0.059000,0.973200,0.751137,0.047624,0.085168,0.137900,0.125000,0.238423,0.067192,0.105885,0.050500,0.009231,0.029387,0.067200,0.061200,0.973200,0.758327,0.045223,0.08057,0.137900,0.12500,0.233854,0.065914,0.110874,0.052600,0.008358,0.028161,0.066600,0.059000,0.973200,0.754344,0.04742,0.084128,0.1379,0.125000,0.237846,0.068069,0.107063,0.051400,0.008979,0.029296,NaN,block of flats,0.039200,"Stone, brick",No,0.0,0.0,0.0,0.0,-1740.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.035792,-18064,-4469,-9118.0,-1623,11.786027,1,1,0,1,0,0,Low-skill Laborers,2.0,2,2,FRIDAY,9,0,0,0,0,0,0,Self-employed,0.564990,0.291656,0.432962,0.122388,0.090065,0.978828,0.751137,0.047624,0.085168,0.151777,0.233706,0.238423,0.067192,0.105885,0.112286,0.009231,0.029387,0.119078,0.088998,0.978292,0.758327,0.045223,0.08057,0.147161,0.22939,0.233854,0.065914,0.110874,0.110687,0.008358,0.028161,0.122809,0.089529,0.978822,0.754344,0.04742,0.084128,0.1512,0.233154,0.237846,0.068069,0.107063,0.113368,0.008979,0.029296,NaN,NaN,0.107129,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,NaN,Working,Higher education,Married,House / apartment,0.019101,-20038,-4458,-2175.0,-3503,5.000000,1,1,0,1,0,0,Drivers,2.0,2,2,MONDAY,14,0,0,0,0,

In [88]:
averages = df_application_test.mean()

In [90]:
df_application_test.head().fillna(value=averages)

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,Unaccompanied,Working,Higher education,Married,House / apartment,0.018850,-19241,-2329,-5170.0,-812,11.786027,1,1,0,1,0,1,NaN,2.0,2,2,TUESDAY,18,0,0,0,0,0,0,Kindergarten,0.752614,0.789654,0.159520,0.066000,0.059000,0.973200,0.751137,0.047624,0.085168,0.137900,0.125000,0.238423,0.067192,0.105885,0.050500,0.009231,0.029387,0.067200,0.061200,0.973200,0.758327,0.045223,0.08057,0.137900,0.12500,0.233854,0.065914,0.110874,0.052600,0.008358,0.028161,0.066600,0.059000,0.973200,0.754344,0.04742,0.084128,0.1379,0.125000,0.237846,0.068069,0.107063,0.051400,0.008979,0.029296,NaN,block of flats,0.039200,"Stone, brick",No,0.0,0.0,0.0,0.0,-1740.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.035792,-18064,-4469,-9118.0,-1623,11.786027,1,1,0,1,0,0,Low-skill Laborers,2.0,2,2,FRIDAY,9,0,0,0,0,0,0,Self-employed,0.564990,0.291656,0.432962,0.122388,0.090065,0.978828,0.751137,0.047624,0.085168,0.151777,0.233706,0.238423,0.067192,0.105885,0.112286,0.009231,0.029387,0.119078,0.088998,0.978292,0.758327,0.045223,0.08057,0.147161,0.22939,0.233854,0.065914,0.110874,0.110687,0.008358,0.028161,0.122809,0.089529,0.978822,0.754344,0.04742,0.084128,0.1512,0.233154,0.237846,0.068069,0.107063,0.113368,0.008979,0.029296,NaN,NaN,0.107129,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,NaN,Working,Higher education,Married,House / apartment,0.019101,-20038,-4458,-2175.0,-3503,5.000000,1,1,0,1,0,0,Drivers,2.0,2,2,MONDAY,14,0,0,0,0,

In [77]:
cleaned_df.iloc[:,valid_columns]

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,AMT_ANNUITY_IS_NULL,AMT_GOODS_PRICE_IS_NULL,NAME_TYPE_SUITE_IS_NULL,OWN_CAR_AGE_IS_NULL,OCCUPATION_TYPE_IS_NULL,CNT_FAM_MEMBERS_IS_NULL,EXT_SOURCE_1_IS_NULL,EXT_SOURCE_2_IS_NULL,EXT_SOURCE_3_IS_NULL,APARTMENTS_AVG_IS_NULL,BASEMENTAREA_AVG_IS_NULL,YEARS_BEGINEXPLUATATION_AVG_IS_NULL,YEARS_BUILD_AVG_IS_NULL,COMMONAREA_AVG_IS_NULL,ELEVATORS_AVG_IS_NULL,ENTRANCES_AVG_IS_NULL,FLOORSMAX_AVG_IS_NULL,FLOORSMIN_AVG_IS_NULL,LANDAREA_AVG_IS_NULL,LIVINGAPARTMENTS_AVG_IS_NULL,LIVINGAREA_AVG_IS_NULL,NONLIVINGAPARTMENTS_AVG_IS_NULL,NONLIVINGAREA_AVG_IS_NULL,APARTMENTS_MODE_IS_NULL,BASEMENTAREA_MODE_IS_NULL,YEARS_BEGINEXPLUATATION_MODE_IS_NULL,YEARS_BUILD_MODE_IS_NULL,COMMONAREA_MODE_IS_NULL,ELEVATORS_MODE_IS_NULL,ENTRANCES_MODE_IS_NULL,FLOORSMAX_MODE_IS_NULL,FLOORSMIN_MODE_IS_NULL,LANDAREA_MODE_IS_NULL,LIVINGAPARTMENTS_MODE_IS_NULL,LIVINGAREA_MODE_IS_NULL,NONLIVINGAPARTMENTS_MODE_IS_NULL,NONLIVINGAREA_MODE_IS_NULL,APARTMENTS_MEDI_IS_NULL,BASEMENTAREA_MEDI_IS_NULL,YEARS_BEGINEXPLUATATION_MEDI_IS_NULL,YEARS_BUILD_MEDI_IS_NULL,COMMONAREA_MEDI_IS_NULL,ELEVATORS_MEDI_IS_NULL,ENTRANCES_MEDI_IS_NULL,FLOORSMAX_MEDI_IS_NULL,FLOORSMIN_MEDI_IS_NULL,LANDAREA_MEDI_IS_NULL,LIVINGAPARTMENTS_MEDI_IS_NULL,LIVINGAREA_MEDI_IS_NULL,NONLIVINGAPARTMENTS_MEDI_IS_NULL,NONLIVINGAREA_MEDI_IS_NULL,FONDKAPREMONT_MODE_IS_NULL,HOUSETYPE_MODE_IS_NULL,TOTALAREA_MODE_IS_NULL,WALLSMATERIAL_MODE_IS_NULL,EMERGENCYSTATE_MODE_IS_NULL,OBS_30_CNT_SOCIAL_CIRCLE_IS_NULL,DEF_30_CNT_SOCIAL_CIRCLE_IS_NULL,OBS_60_CNT_SOCIAL_CIRCLE_IS_NULL,DEF_60_CNT_SOCIAL_CIRCLE_IS_NULL,DAYS_LAST_PHONE_CHANGE_IS_NULL,AMT_REQ_CREDIT_BUREAU_HOUR_IS_NULL,AMT_REQ_CREDIT_BUREAU_DAY_IS_NULL,AMT_REQ_CREDIT_BUREAU_WEEK_IS_NULL,AMT_REQ_CREDIT_BUREAU_MON_IS_NULL,AMT_REQ_CREDIT_BUREAU_QRT_IS_NULL,AMT_REQ_CREDIT_BUREAU_YEAR_IS_NULL,DUM__Cash loans,DUM__Revolving loans,DUM__F,DUM__M,DUM__N,DUM__Y,DUM__N,DUM__Y,DUM__Children,DUM__Family,DUM__Group of people,DUM__Other_A,DUM__Other_B,"DUM__Spouse, partner",DUM__Unaccompanied,DUM__Businessman,DUM__Commercial associate,DUM__Pensioner,DUM__State servant,DUM__Student,DUM__Unemployed,DUM__Working,D

In [38]:
cleaned_df.iloc[:,valid_columns]

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,AMT_ANNUITY_IS_NULL,AMT_GOODS_PRICE_IS_NULL,NAME_TYPE_SUITE_IS_NULL,OWN_CAR_AGE_IS_NULL,OCCUPATION_TYPE_IS_NULL,CNT_FAM_MEMBERS_IS_NULL,EXT_SOURCE_1_IS_NULL,EXT_SOURCE_2_IS_NULL,EXT_SOURCE_3_IS_NULL,APARTMENTS_AVG_IS_NULL,BASEMENTAREA_AVG_IS_NULL,YEARS_BEGINEXPLUATATION_AVG_IS_NULL,YEARS_BUILD_AVG_IS_NULL,COMMONAREA_AVG_IS_NULL,ELEVATORS_AVG_IS_NULL,ENTRANCES_AVG_IS_NULL,FLOORSMAX_AVG_IS_NULL,FLOORSMIN_AVG_IS_NULL,LANDAREA_AVG_IS_NULL,LIVINGAPARTMENTS_AVG_IS_NULL,LIVINGAREA_AVG_IS_NULL,NONLIVINGAPARTMENTS_AVG_IS_NULL,NONLIVINGAREA_AVG_IS_NULL,APARTMENTS_MODE_IS_NULL,BASEMENTAREA_MODE_IS_NULL,YEARS_BEGINEXPLUATATION_MODE_IS_NULL,YEARS_BUILD_MODE_IS_NULL,COMMONAREA_MODE_IS_NULL,ELEVATORS_MODE_IS_NULL,ENTRANCES_MODE_IS_NULL,FLOORSMAX_MODE_IS_NULL,FLOORSMIN_MODE_IS_NULL,LANDAREA_MODE_IS_NULL,LIVINGAPARTMENTS_MODE_IS_NULL,LIVINGAREA_MODE_IS_NULL,NONLIVINGAPARTMENTS_MODE_IS_NULL,NONLIVINGAREA_MODE_IS_NULL,APARTMENTS_MEDI_IS_NULL,BASEMENTAREA_MEDI_IS_NULL,YEARS_BEGINEXPLUATATION_MEDI_IS_NULL,YEARS_BUILD_MEDI_IS_NULL,COMMONAREA_MEDI_IS_NULL,ELEVATORS_MEDI_IS_NULL,ENTRANCES_MEDI_IS_NULL,FLOORSMAX_MEDI_IS_NULL,FLOORSMIN_MEDI_IS_NULL,LANDAREA_MEDI_IS_NULL,LIVINGAPARTMENTS_MEDI_IS_NULL,LIVINGAREA_MEDI_IS_NULL,NONLIVINGAPARTMENTS_MEDI_IS_NULL,NONLIVINGAREA_MEDI_IS_NULL,FONDKAPREMONT_MODE_IS_NULL,HOUSETYPE_MODE_IS_NULL,TOTALAREA_MODE_IS_NULL,WALLSMATERIAL_MODE_IS_NULL,EMERGENCYSTATE_MODE_IS_NULL,OBS_30_CNT_SOCIAL_CIRCLE_IS_NULL,DEF_30_CNT_SOCIAL_CIRCLE_IS_NULL,OBS_60_CNT_SOCIAL_CIRCLE_IS_NULL,DEF_60_CNT_SOCIAL_CIRCLE_IS_NULL,DAYS_LAST_PHONE_CHANGE_IS_NULL,AMT_REQ_CREDIT_BUREAU_HOUR_IS_NULL,AMT_REQ_CREDIT_BUREAU_DAY_IS_NULL,AMT_REQ_CREDIT_BUREAU_WEEK_IS_NULL,AMT_REQ_CREDIT_BUREAU_MON_IS_NULL,AMT_REQ_CREDIT_BUREAU_QRT_IS_NULL,AMT_REQ_CREDIT_BUREAU_YEAR_IS_NULL
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018850,-19241,-2329,-5170.0,-812,NaN,1,1,0,1,0,1,2.0,2,2,18,0,0,0,0,0,0,0.752614,0.789654,0.159520,0.0660,0.0590,0.9732,NaN,NaN,NaN,0.1379,0.1250,NaN,NaN,NaN,0.0505,NaN,NaN,0.0672,0.0612,0.9732,NaN,NaN,NaN,0.1379,0.1250,NaN,NaN,NaN,0.0526,NaN,NaN,0.0666,0.0590,0.973

In [31]:
predictions = model.predict(cleaned_df.iloc[:,valid_columns])

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [29]:
X_train, X_test, y_train, y_test = train_test_split(relevant_x_df, target_y, test_size=0.33, random_state=42)

In [30]:
model = RandomForestClassifier()

model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [14]:
plot_roc(model, X_test, y_test)